In [ ]:
import ee
import datetime
import pandas as pd

ee.Authenticate()
ee.Initialize()

def get_ndvi(lat, lon, date):
    # Convert the date string to a datetime object
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    start_date = date - datetime.timedelta(days=365)

    point = ee.Geometry.Point([lon, lat])

    # Load the MODIS Terra Vegetation Indices dataset
    dataset = ee.ImageCollection('MODIS/006/MOD13A1') \
        .filterDate(ee.Date(start_date), ee.Date(date)) \
        .sort('system:time_start', opt_ascending=False) \
        .first()

    if dataset is None:
        return None

    ndvi = dataset.select('NDVI')

    # Get the NDVI value for the point
    ndvi_samples = ndvi.sample(point, 500)

    if ndvi_samples.size().getInfo() == 0:
        return None

    ndvi_value = ndvi_samples.first().get('NDVI').getInfo()
    return ndvi_value

import csv
import pandas as pd

# Function to save the results to a CSV file
def save_to_csv(filename, data):
    with open(filename, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Latitude', 'Longitude', 'Date', 'NDVI'])

        for row in data:
            writer.writerow(row)


            
# Read the CSV file
df = pd.read_csv("mapdata_wildfires.csv")

coordinates_and_dates = []


for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    date_created = row['incident_dateonly_created']

    # Create a tuple and append it to the list
    coordinates_and_dates.append((lat, lon, date_created))

results = []

# Iterate through the coordinate and date pairs and get the NDVI values
for lat, lon, date in coordinates_and_dates:
    ndvi = get_ndvi(lat, lon, date)
    print(f'Latitude: {lat}, Longitude: {lon}, Date: {date}, NDVI: {ndvi}')
    results.append((lat, lon, date, ndvi))

# Save the results to a CSV file
save_to_csv('vegetation_indices_data_wildfires.csv', results)


In [ ]:
print("The results were saved to a CSV file.")